### КР2

### Бустинг. - 6 баллов
В существующий код бустинга добавьте возможность ранней остановки обучения.
должны быть учтены:
1) Наличие валидационного датасета (либо разделение должно быть внутри класса, либо вне его, а в обучении новый набор будет подаваться отдельной парой)
2) Кастомная метрика или лосс для оствновки. Должна передаваться в виде доп. параметра. Дефолт - лосс функция для расчета градиента.
3) Укажите, сколько должно пройти итераций для ранней остановки.
4) После обучения должно вернуться лучшее состояние модели по валидационной выборке, а не то, которое было достинуто при остановке обучения.

Для обучения используйте тот же датасет, что использовался на 8 семинаре (house_price_regression_dataset).
1 и 3 пункты обязательны - 3 балла. 2 пункт - 1 балл (при недефолтной реализации). 4 пункт - 2 балла.

In [84]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np
### Собственная реализация
class MyGradientRegressor:
    def __init__(self, n_estimators: int = 300, max_depth: int = 3, lr: float = 0.1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.lr = lr
        self.estimators = []

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train)

        self.estimators = []
        predictions = 0

        for _ in range(self.n_estimators):
            new_model = DecisionTreeRegressor(max_depth=self.max_depth)
            new_target = -2 * (predictions - y_train)
            new_model.fit(X_train, new_target)
            predictions += self.lr * new_model.predict(X_train)
            self.estimators.append(new_model)

    def predict(self, X_test):
        X_test = np.array(X_test)
        curr_pred = 0
        for est in self.estimators:
            curr_pred += self.lr * est.predict(X_test)

        return curr_pred

In [85]:
class MyNewGradientRegressor:
    def __init__(self, n_estimators: int = 300, max_depth: int = 3, lr: float = 0.1,
                 stop: int = 10, scoring_metric=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.lr = lr
        self.estimators = []
        self.best_estimator = None
        self.stop = stop
        if scoring_metric is not None:
          self.scoring_metric = scoring_metric
        else: self.scoring_metric = self.loss
        self.best_score = 0

    def loss(self, y, y_pred):
        return np.mean((y - y_pred) ** 2)

    def fit(self, X_train, y_train, X_val=None, y_val=None):
        X_train = np.array(X_train)
        y_train = np.array(y_train)

        self.estimators = []
        predictions = np.zeros(y_train.shape)
        stop_count = 0

        for i in range(self.n_estimators):
            new_model = DecisionTreeRegressor(max_depth=self.max_depth)
            new_target = -2 * (predictions - y_train)
            new_model.fit(X_train, new_target)
            predictions += self.lr * new_model.predict(X_train)
            self.estimators.append(new_model)

            if X_val is not None and y_val is not None:
                X_val = np.array(X_val)
                y_val = np.array(y_val)
                val_predictions = self.predict(X_val)
                score = self.scoring_metric(y_val, val_predictions)

                if score < self.best_score:
                    self.best_score = score
                    self.best_estimator = new_model
                    stop_count = 0
                else:
                    stop_count += 1
                if stop_count >= self.stop:
                    break

    def predict(self, X_test):
        X_test = np.array(X_test)
        curr_pred = np.zeros(X_test.shape[0])
        for est in self.estimators:
            curr_pred += self.lr * est.predict(X_test)

        return curr_pred

In [86]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv("house_price_regression_dataset.csv")

X = data.drop(columns=["House_Price"])
y = data["House_Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)
y_train = y_train.reset_index()["House_Price"]
y_test = y_test.reset_index()["House_Price"]

Старый вариант

In [87]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
my_model = MyGradientRegressor(n_estimators=300, max_depth=3, lr=0.1)
my_model.fit(X_train, y_train)

pred = my_model.predict(X_test)
mean_absolute_error(y_test, pred)

12370.582471045378

Новый вариант

In [88]:
my_model = MyNewGradientRegressor(n_estimators=300, max_depth=3, lr=0.1, stop=50)
my_model.fit(X_train, y_train, X_val=X_test, y_val=y_test)

pred = my_model.predict(X_test)
mean_absolute_error(y_test, pred)

12859.19161059405

### Стекинг - 4 балла
В текущей реализации в качестве признаков для метамодели используются предсказания базовых моделей.
Ваша задача добавить возможность дополнительно учитывать исходные данные в качестве признаков (гиперпараметр).
Метапризнаки как доп. фичи к основным.
При этом на основные признаки добавляется воможность расчета полиномиальных признаков (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

Для тестирования используйте тот же датасет

In [89]:
class Stacking:
    def __init__(self, estimators, meta_estimator, folds=5):
        self.estimators = estimators
        self.meta_estimator = meta_estimator
        self.folds = folds
        self.meta_train = []

    def _fit_estimator(self, estimator, X_train, y_train):
        kf = KFold(n_splits=self.folds, shuffle=True)
        train_fold_indices = []
        test_fold_indices = []
        test_fold_predicts = []

        for train_idx, test_idx in kf.split(X_train):
            train_fold_indices.extend(train_idx)
            test_fold_indices.extend(test_idx)

            estimator.fit(X_train[train_idx], y_train[train_idx])
            test_fold_predicts.extend(estimator.predict(X_train[test_idx]))

        estimator.fit(X_train, y_train)
        self.meta_train.append(np.array(test_fold_predicts)[np.argsort(test_fold_indices)])

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        self.meta_train = []

        for estimator in self.estimators:
            self._fit_estimator(estimator, X_train, y_train)

        self.meta_train = np.array(self.meta_train).transpose()
        self.meta_estimator.fit(self.meta_train, y_train)

    def predict(self, X_test):
        X_test = np.array(X_test)
        meta_features = np.array([estimator.predict(X_test) for estimator in self.estimators]).transpose()
        return self.meta_estimator.predict(meta_features)

In [90]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures

class NewStacking:
    def __init__(self, estimators, meta_estimator, folds=5, use_features=False, degree=1):
        self.estimators = estimators
        self.meta_estimator = meta_estimator
        self.folds = folds
        self.use_features = use_features
        self.degree = degree
        self.meta_train = []

    def _fit_estimator(self, estimator, X_train, y_train):
        kf = KFold(n_splits=self.folds, shuffle=True)
        train_fold_indices = []
        test_fold_indices = []
        test_fold_predicts = []

        for train_idx, test_idx in kf.split(X_train):
            train_fold_indices.extend(train_idx)
            test_fold_indices.extend(test_idx)

            estimator.fit(X_train[train_idx], y_train[train_idx])
            test_fold_predicts.extend(estimator.predict(X_train[test_idx]))

        estimator.fit(X_train, y_train)
        self.meta_train.append(np.array(test_fold_predicts)[np.argsort(test_fold_indices)])

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        self.meta_train = []

        if self.use_features:
            poly = PolynomialFeatures(degree=self.degree)
            X_train_poly = poly.fit_transform(X_train)
        else:
            X_train_poly = X_train

        for estimator in self.estimators:
            self._fit_estimator(estimator, X_train_poly, y_train)

        self.meta_train = np.array(self.meta_train).transpose()
        self.meta_estimator.fit(self.meta_train, y_train)

    def predict(self, X_test):
        X_test = np.array(X_test)

        if self.use_features:
            poly = PolynomialFeatures(degree=self.degree)
            X_test_poly = poly.fit_transform(X_test)
        else:
            X_test_poly = X_test

        meta_features = np.array([estimator.predict(X_test_poly) for estimator in self.estimators]).transpose()
        return self.meta_estimator.predict(meta_features)


In [93]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, train_test_split
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

data = pd.read_csv("house_price_regression_dataset.csv")

X = data.drop(columns=["House_Price"])
y = data["House_Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)
y_train = y_train.reset_index()["House_Price"]
y_test = y_test.reset_index()["House_Price"]

Старый вариант

In [94]:
model = Stacking(
    estimators=[
        LinearRegression(),
        DecisionTreeRegressor(max_depth=3),
        XGBRegressor(n_estimators=100, max_depth=3),
    ],
    meta_estimator=LinearRegression(),
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean_absolute_error(y_test, y_pred)

7830.853761245918

Новый вариант

In [95]:
model = NewStacking(
    estimators=[
        LinearRegression(),
        DecisionTreeRegressor(max_depth=3),
        XGBRegressor(n_estimators=100, max_depth=3),
    ],
    meta_estimator=LinearRegression(),
    use_features=True,
    degree=2
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean_absolute_error(y_test, y_pred)

8028.888204153737